<p style="font-family:verdana;font-size:200%;text-align:center;">Logistic Regression</p>

### 실습 데이터셋 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# 인터넷으로 공유 중인 텍스트 데이터를 읽고 데이터프레임 df를 생성합니다.
df = pd.read_csv('https://bit.ly/university_admit')

In [ ]:
# df의 정보를 확인합니다.
df.info()

### 기술통계량 확인

In [ ]:
# 정수형 변수 rank를 문자형으로 변환합니다.
df['rank'] = df['rank'].astype('str')

In [ ]:
# 연속형 변수의 기술통계량을 출력합니다.
df.describe().round(2)

In [ ]:
# 범주형 변수의 기술통계량을 출력합니다.
df.describe(include = 'object')

In [ ]:
# rank의 범주별 빈도수를 출력합니다.
df['rank'].value_counts().sort_index()

### 목표변수 시각화

In [ ]:
# 관련 모듈을 호출합니다.
from GraphicSetting import *
import HelloDataScience as hds

In [ ]:
# 목표변수의 범주별 빈도수로 데이터프레임을 생성합니다.
freq = df.groupby(by = ['admit']).count()[['gre']]

In [ ]:
# freq의 열이름을 변경하고, 인덱스를 초기화합니다.
freq = freq.rename(columns = {'gre': 'freq'}).reset_index()

In [ ]:
# freq를 출력합니다.
freq

In [ ]:
# 목표변수의 범주별 빈도수로 막대그래프를 그립니다.
sns.barplot(data = freq, x = 'admit', y = 'freq')
for index, row in freq.iterrows():
    plt.text(x = index, y = row['freq'] + 5, s = row['freq'], 
             fontsize = 12, ha = 'center', va = 'bottom', c = 'black')
plt.title(label = '목표변수의 범주별 빈도수 비교')
plt.ylim(0, 300);

### 연속형 입력변수와 관계 파악

In [ ]:
# admit의 범주별 gre의 분포를 비교합니다.
hds.plot_box_group(data = df, x = 'admit', y = 'gre')

In [ ]:
# admit의 범주별 gpa의 분포를 비교합니다.
hds.plot_box_group(data = df, x = 'admit', y = 'gpa')

### 범주형 입력변수와 관계 파악

In [ ]:
# rank의 범주별 admit의 빈도수를 막대그래프로 시각화합니다.
sns.countplot(data = df, x = 'rank', hue = 'admit', order = '1234')
plt.title(label = 'admit 범주별 rank의 빈도수 비교');

### t-검정: gre

In [ ]:
# admit의 범주별 gre 평균을 갖는 데이터프레임 avg를 생성합니다.
avg = df.groupby(by = ['admit']).mean()[['gre']].reset_index()
avg = avg.rename(columns = {'gre': 'greMean'})

In [ ]:
# df에서 목표변수와 입력변수를 선택하고, avg와 병합하여 mgd를 생성합니다.
mgd = pd.merge(left = df[['admit', 'gre']], right = avg)
mgd['residual'] = mgd['gre'] - mgd['greMean']

In [ ]:
# residual의 정규성 검정을 실행합니다.
stats.shapiro(x = mgd['residual'])

In [ ]:
# 등분산성 검정을 위해 범주형 변수 기준으로 샘플 데이터셋을 생성합니다.
sample1 = df.loc[df['admit'] == 0, 'gre']
sample2 = df.loc[df['admit'] == 1, 'gre']

In [ ]:
# (정규성 가정 만족) 등분산성 검정을 실행합니다.
stats.bartlett(sample1, sample2)

In [ ]:
# (정규성 가정 만족) 등분산 가정된 t-검정을 실행합니다.
stats.ttest_ind(sample1, sample2, equal_var = True)

In [ ]:
# (정규성 가정 불만족) 윌콕슨 순위합 검정을 실행합니다.
stats.ranksums(sample1, sample2)

### t-검정 : gpa

In [ ]:
# admit의 범주별 gpa 평균을 갖는 데이터프레임 avg를 생성합니다.
avg = df.groupby(by = ['admit']).mean()[['gpa']].reset_index()
avg = avg.rename(columns = {'gpa': 'gpaMean'})

In [ ]:
# df에서 목표변수와 입력변수를 선택하고, avg와 병합하여 mgd를 생성합니다.
mgd = pd.merge(left = df[['admit', 'gpa']], right = avg)
mgd['residual'] = mgd['gpa'] - mgd['gpaMean']

In [ ]:
# residual의 정규성 검정을 실행합니다.
stats.shapiro(x = mgd['residual'])

In [ ]:
# 등분산성 검정을 위해 범주형 변수 기준으로 샘플 데이터셋을 생성합니다.
sample1 = df.loc[df['admit'] == 0, 'gpa']
sample2 = df.loc[df['admit'] == 1, 'gpa']

In [ ]:
# (정규성 가정 만족) 등분산성 검정을 실행합니다.
stats.bartlett(sample1, sample2)

In [ ]:
# (정규성 가정 만족) 등분산 가정된 t-검정을 실행합니다.
stats.ttest_ind(sample1, sample2, equal_var = True)

In [ ]:
# (정규성 가정 불만족) 윌콕슨 순위합 검정을 실행합니다.
stats.ranksums(sample1, sample2)

### 교차분석: rank

In [ ]:
# 범주형 입력변수 rank와 목표변수의 교차테이블을 출력합니다.
cross = pd.crosstab(index = df['rank'], columns = df['admit'])

In [ ]:
# 교차테이블을 출력합니다.
cross

In [ ]:
# 교차테이블 빈도수로 카이제곱 검정을 실행합니다.
stats.chi2_contingency(observed = cross)

### 더미변수 생성

In [ ]:
# 범주형 입력변수로 더미변수를 생성합니다.
dm = pd.get_dummies(data = df['rank'], prefix = 'rank', drop_first = True)

In [ ]:
# df의 오른쪽에 더미변수 dm을 추가합니다.
df = pd.concat(objs = [df, dm], axis = 1)

In [ ]:
# df의 처음 10행을 출력합니다.
df.head(n = 10)

In [ ]:
# df에서 rank를 삭제합니다.
df = df.drop(labels = ['rank'], axis = 1)

### 실습 데이터셋 분할

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.model_selection import train_test_split

In [ ]:
# 목표변수 열이름을 지정합니다.
yvar = 'admit'

In [ ]:
# 입력변수 행렬 X와 목표변수 벡터 y를 생성합니다.
X = df.drop(labels = [yvar], axis = 1)
y = df[yvar]

In [ ]:
# 전체 데이터의 70%를 훈련셋, 30%를 시험셋으로 분할합니다.
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size = 0.3, random_state = 0
)

In [ ]:
# 훈련셋의 목표변수 범주별 상대도수를 확인합니다.
y_tr.value_counts(normalize = True)

In [ ]:
# 시험셋의 목표변수 범주별 상대도수를 확인합니다.
y_te.value_counts(normalize = True)

### 로지스틱 회귀모형 적합

In [ ]:
# 관련 라이브러리를 호출합니다.
import statsmodels.api as sm

In [ ]:
# 훈련셋 입력변수에 y절편의 역할을 수행할 상수 1을 추가합니다.
X_tr = sm.add_constant(data = X_tr)

In [ ]:
# 로지스틱 회귀모형을 반환하는 함수를 정의합니다.
def glm(y, X):
    model = sm.GLM(endog = y, exog = X, family = sm.families.Binomial())
    return model.fit()

### 로지스틱 회귀모형 적합 결과 확인

In [ ]:
# 훈련셋으로 로지스틱 회귀모형 적합 결과를 확인합니다.
fit1 = glm(y = y_tr, X = X_tr)
fit1.summary()

### 로지스틱 회귀모형의 유의성 검정

In [ ]:
# 두 모형의 이탈도 차이를 생성합니다.
devGap = fit1.null_deviance - fit1.deviance
devGap

In [ ]:
# 두 모형의 자유도 차이를 생성합니다.
dfGap = fit1.df_model
dfGap

In [ ]:
# 검정통계량과 자유도로 유의확률을 출력합니다.
1 - stats.chi2.cdf(x = devGap, df = dfGap)

### 다중공선성 확인

In [ ]:
# 관련 모듈을 호출합니다.
import HelloDataScience as hds

In [ ]:
# 훈련셋의 입력변수별 분산팽창지수를 출력합니다.
hds.vif(X = X_tr)

### 입력변수의 오즈비 확인

In [ ]:
# 입력변수별 회귀계수의 오즈비를 확인합니다.
np.exp(fit1.params).round(4)

### 표준화 회귀계수 확인

In [ ]:
# 로지스틱 회귀모형의 회귀계수만 출력합니다.
fit1.params

In [ ]:
# 입력변수의 표준편차를 목표변수의 표준편차로 나눈 값을 출력합니다.
# 로짓변환된 목표변수는 표준편차를 구할 수 없으므로 상수 1로 대신합니다.
X_tr.std() / 1

In [ ]:
# 표준화 회귀계수를 생성합니다.
beta_z = fit1.params * X_tr.std()

In [ ]:
# 표준화 회귀계수의 절대값을 오름차순으로 정렬하여 출력합니다.
beta_z.abs().sort_values()

### 목표변수의 추정확률 생성

In [ ]:
# 훈련셋과의 추정확률을 생성합니다.
y_tr_prob = fit1.predict(exog = X_tr)

In [ ]:
# 훈련셋의 추정확률 일부를 출력합니다.
y_tr_prob[0:5]

In [ ]:
# 훈련셋의 실제값과 비교합니다.
y_tr[0:5]

In [ ]:
# 시험셋에 상수항을 추가합니다.
X_te = sm.add_constant(data = X_te)

In [ ]:
# 시험셋의 추정확률을 생성합니다.
y_te_prob = fit1.predict(exog = X_te)

In [ ]:
# 시험셋의 추정확률 일부를 출력합니다.
y_te_prob[0:5]

In [ ]:
# 시험셋의 실제값과 비교합니다.
y_te[0:5]

### 분류모형 성능 평가 : ROC 곡선

In [ ]:
# fit1의 추정확률로 ROC 곡선과 AUC를 출력합니다.
hds.plot_roc(
    y1_true = y_tr, 
    y1_prob = y_tr_prob, 
    y2_true = y_te, 
    y2_prob = y_te_prob, 
    pos = 1
)

### 목표변수의 추정값 생성

In [ ]:
# 분리 기준점을 0.5로 설정합니다.
cutoff = 0.5

In [ ]:
# 훈련셋의 목표변수 추정값(라벨)을 생성합니다.
y_tr_pred = pd.Series(np.where(y_tr_prob >= cutoff, 1, 0))

In [ ]:
# 훈련셋의 추정값 빈도수를 확인합니다.
y_tr_pred.value_counts()

In [ ]:
# 시험셋의 목표변수 추정값(라벨)을 생성합니다.
y_te_pred = pd.Series(np.where(y_te_prob >= cutoff, 1, 0))

In [ ]:
# 시험셋의 추정값 빈도수를 확인합니다.
y_te_pred.value_counts()

### 분류모형 성능 평가 : 혼동행렬 리포트

In [ ]:
# 관련 라이브러리를 호출합니다.
from sklearn.metrics import *

In [ ]:
# 훈련셋의 혼동행렬을 출력합니다.
confusion_matrix(y_true = y_tr, y_pred = y_tr_pred)

In [ ]:
# 훈련셋의 혼동행렬 리포트를 출력합니다.
print(classification_report(y_true = y_tr, y_pred = y_tr_pred))

In [ ]:
# 시험셋의 혼동행렬을 출력합니다.
confusion_matrix(y_true = y_te, y_pred = y_te_pred)

In [ ]:
# 시험셋의 혼동행렬 리포트를 출력합니다.
print(classification_report(y_true = y_te, y_pred = y_te_pred))

<p style="font-family:verdana;font-size:200%;text-align:center;">End of Document</p>